[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/langchain-course/blob/main/chapters/08-streaming.ipynb)

#### LangChain Essentials Course

# Streaming With Langchain

LangChain is one of the most popular open source libraries for AI Engineers. It's goal is to abstract away the complexity in building AI software, provide easy-to-use building blocks, and make it easier when switching between AI service providers.

In this example, we will introduce LangChain's async streaming, allowing us to receive and view the tokens as they are generated by OpenAI's LLM. The use of streaming is typical in conversational interfaces and can provide a more natural experience for users.

In [1]:
!pip install -qU \
  langchain-core==0.3.33 \
  langchain-openai==0.3.3 \
  langchain-community==0.3.16 \
  langsmith==0.3.4 \
  langchain-ollama


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 954.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.3/333.3 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00


---

> ⚠️ We will be using OpenAI for this example allowing us to run everything via API. If you would like to use Ollama instead, check out the [Ollama LangChain Course](https://github.com/aurelio-labs/langchain-course/tree/main/notebooks/ollama).

---

In [2]:

from google.colab import userdata
Lang_api =userdata.get('Langsmith_api')

---

> ⚠️ If using LangSmith, add your API key below:

In [3]:
import os
from getpass import getpass

# must enter API key
os.environ["LANGCHAIN_API_KEY"] =Lang_api
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "aurelioai-langchain-course-streaming-openai"

In [5]:
!apt update && apt install -y pciutils lshw

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,237 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,692 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packa

---

In [32]:
! curl -fsSL https://ollama.ai/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [33]:
!ollama serve > /dev/null 2>&1 &

In [34]:

!ollama pull mistral

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling ff82381e2bea... 100% ▕▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕▏  11 KB                         
pulling 491dfa501e59... 100% ▕▏  801 B                         
pulling ed11eda7790d... 100% ▕▏   30 B                         
pulling 42347cd80dc8... 100% ▕▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [35]:
!ollama list

NAME              ID              SIZE      MODIFIED      
mistral:latest    f974a74358d6    4.1 GB    3 seconds ago    


In [9]:

import os
from langchain_ollama.chat_models import ChatOllama

model_name = "mistral"

# initialize one LLM with temperature 0.0, this makes the LLM more deterministic
llm = ChatOllama(temperature=0.0, model=model_name)
creative_llm = ChatOllama(temperature=0.9, model=model_name)

For the LLM, we'll start by initializing our connection to the OpenAI API. We do need an OpenAI API key, which you can get from the [OpenAI platform](https://platform.openai.com/api-keys).

We will use the `gpt-4o-mini` model with a `temperature` of `0.0`:

In [10]:
# import os
# from getpass import getpass
# from langchain_openai import ChatOpenAI

# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") \
#     or getpass("Enter your OpenAI API key: ")

# llm = ChatOpenAI(
#     model_name="gpt-4o",
#     temperature=0.0,
#     streaming=True
# )

In [11]:
# llm_out = llm.invoke("Hello there")
# llm_out

## Streaming with `astream`

We will start by creating a aysnc stream from our LLM. We do this within an `async for` loop, allowing us to iterate through the chunks of data and use them as soon as the async `astream` method returns the tokens to us. By adding a pipe character `|` we can see the individual tokens that are generated. We set `flush` equal to `True` as this forces immediate output to the console, resulting in smoother streaming.

In [12]:
tokens = []
async for token in llm.astream("What is NLP?"):
    tokens.append(token)
    print(token.content, end="|", flush=True)

 N|LP| stands| for| Natural| Language| Process|ing|.| It| is| a| field| of| study| in| artificial| intelligence| (|AI|)| and| comput|ational| lingu|istics| that| focuses| on| the| interaction| between| computers| and| humans| using| natural| language|.| The| goal| of| N|LP| is| to| enable| computers| to| understand|,| interpret|,| and| generate| human| language| in| a| valuable| way|.| This| includes| tasks| such| as| speech| recognition|,| text|-|to|-|spe|ech| synt|hesis|,| sentiment| analysis|,| machine| translation|,| and| more|.| N|LP| algorithms| use| statistical| models|,| machine| learning| techniques|,| and| deep| learning| architect|ures| to| analyze| and| process| large| amounts| of| natural| language| data|.||

Since we appended each token to the `tokens` list, we can also see what is inside each and every token.

In [10]:
tokens[0]

AIMessageChunk(content=' N', additional_kwargs={}, response_metadata={}, id='run-e5ca200d-4269-47f5-8188-f416ae690602')

In [11]:
tokens[1]

AIMessageChunk(content='LP', additional_kwargs={}, response_metadata={}, id='run-e5ca200d-4269-47f5-8188-f416ae690602')

We can also merge multiple `AIMessageChunk` objects together with the `+` operator, creating a larger set of tokens / chunk:

In [12]:
tokens[0] + tokens[1] + tokens[2] + tokens[3] + tokens[4]

AIMessageChunk(content=' NLP stands for Natural', additional_kwargs={}, response_metadata={}, id='run-e5ca200d-4269-47f5-8188-f416ae690602')

A word of caution, there is nothing preventing you from merging tokens in the incorrect order, so be cautious to not output any token omelettes:

In [13]:
tokens[4] + tokens[3] + tokens[2] + tokens[1] + tokens[0]

AIMessageChunk(content=' Natural for standsLP N', additional_kwargs={}, response_metadata={}, id='run-e5ca200d-4269-47f5-8188-f416ae690602')

## Streaming with Agents

Streaming with agents, particularly the custom agent executor, is a little more complex. Let's begin by constructor a simple agent executor matching what we built in the [Agent Executor](https://github.com/aurelio-labs/langchain-course/blob/main/notebooks/openai/07-custom-agent-executor.ipynb) chapter.

To construct the agent executor we need:

* Tools
* `ChatPromptTemplate`
* Our LLM (already defined with `llm`)
* An agent
* Finally, the agent executor

Let's start defining each.

### Tools

Now we will define a few tools to be used by an async agent executor. Our goal for tool-use in regards to streaming are:

* The tool-use steps will be streamed in one big chunk, ie we do not return the tool use information token-by-token but instead it streams message-by-message.

* The final LLM output _will_ be streamed token-by-token as we saw above.

For these we need to define a few math tools and our final answer tool.

In [13]:
from langchain_core.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x

@tool
def final_answer(answer: str, tools_used: list[str]) -> str:
    """Use this tool to provide a final answer to the user.
    The answer should be in natural language as this will be provided
    to the user directly. The tools_used must include a list of tool
    names that were used within the `scratchpad`. You MUST use this tool
    to conclude the interaction.
    """
    return {"answer": answer, "tools_used": tools_used}

We'll need all of our tools in a list when defining our `agent` and `agent_executor`.

In [14]:
tools = [add, multiply, exponentiate, subtract, final_answer]

### `ChatPromptTemplate`

We will create our `ChatPromptTemplate`, using a system message, chat history, user input, and a scratchpad for intermediate steps.

In [41]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You should use the tools provided even if you are confident about your answer."
        "You're a helpful assistant. When answering a user's question "
        "you should first use one of the tools provided. After using a "
        "tool the tool output will be provided back to you. You MUST "
        "then use the final_answer tool to provide a final answer to the user. "
        "DO NOT use the same tool more than once."
        "Use the Final answer tool to conclude the interaction. "
        "Using the final answer tool is a must."

    )),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

### Agent

As before, we will define our `agent` with LCEL.

In [45]:
from langchain_core.runnables.base import RunnableSerializable

tools = [add, subtract, multiply, exponentiate, final_answer]

# define the agent runnable
agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any")
)

### Agent Executor

Finally, we will create the agent executor.

In [46]:
import json
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage


# create tool name to function mapping
name2tool = {tool.name: tool.func for tool in tools}

class CustomAgentExecutor:
    chat_history: list[BaseMessage]

    def __init__(self, max_iterations: int = 3):
        self.chat_history = []
        self.max_iterations = max_iterations
        self.agent: RunnableSerializable = (
            {
                "input": lambda x: x["input"],
                "chat_history": lambda x: x["chat_history"],
                "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
            }
            | prompt
            | llm.bind_tools(tools, tool_choice="required")  # we're forcing tool use again
        )

    def invoke(self, input: str) -> dict:
        # invoke the agent but we do this iteratively in a loop until
        # reaching a final answer
        count = 0
        agent_scratchpad = []
        while count < self.max_iterations:
            # invoke a step for the agent to generate a tool call
            out = self.agent.invoke({
                "input": input,
                "chat_history": self.chat_history,
                "agent_scratchpad": agent_scratchpad
            })
            # if the tool call is the final answer tool, we stop
            print(out.tool_calls)
            if len(out.tool_calls)==0:
              count += 1
              continue
            if out.tool_calls[0]["name"] == "final_answer":
                break
            agent_scratchpad.append(out)  # add tool call to scratchpad
            # otherwise we execute the tool and add it's output to the agent scratchpad
            tool_out = name2tool[out.tool_calls[0]["name"]](**out.tool_calls[0]["args"])
            # add the tool output to the agent scratchpad
            action_str = f"The {out.tool_calls[0]['name']} tool returned {tool_out}"
            agent_scratchpad.append({
                "role": "tool",
                "content": action_str,
                "tool_call_id": out.tool_calls[0]["id"]
            })
            # add a print so we can see intermediate steps
            print(f"{count}: {action_str}")
            count += 1
        # add the final output to the chat history
        final_answer = out.tool_calls[0]["args"]
        # this is a dictionary, so we convert it to a string for compatibility with
        # the chat history
        final_answer_str = json.dumps(final_answer)
        self.chat_history.append({"input": input, "output": final_answer_str})
        self.chat_history.extend([
            HumanMessage(content=input),
            AIMessage(content=final_answer_str)
        ])
        # return the final answer in dict form
        return final_answer

agent_executor = CustomAgentExecutor()

Our `agent_executor` is now ready to use, let's quickly test it before adding streaming.

In [48]:
agent_executor.invoke(input="What is 5675657*687687")

[]
[{'name': 'multiply', 'args': {'x': 5675657, 'y': 687687}, 'id': '1c6a2cdd-56c8-40ad-b483-aefa44f3a303', 'type': 'tool_call'}, {'name': 'final_answer', 'args': {'answer': 'The product of 5675657 and 687687 is 39402188789.'}, 'id': '681444f1-15d7-400f-bd5f-2834df7a1257', 'type': 'tool_call'}]
1: The multiply tool returned 3903075535359
[{'name': 'final_answer', 'args': {'answer': 'The product of 5675657 and 687687 is 3903075535359.'}, 'id': '05d0cec5-7d7f-4580-a6c6-fb75841bdf57', 'type': 'tool_call'}]


{'answer': 'The product of 5675657 and 687687 is 3903075535359.'}

Let's modify our `agent_executor` to use streaming and parse the streamed output into a format that we can more easily work with.

First, when streaming with our custom agent executor we will need to pass our callback handler to the agent on every new invocation. To make this simpler we can make the `callbacks` field a configurable field and this will allow us to initialize the agent using the `with_config` method, allowing us to pass the callback handler to the agent with every invocation.

In [49]:
from langchain_core.runnables import ConfigurableField

llm = ChatOllama(
    model="mistral",
    temperature=0.0,
    streaming=True
).configurable_fields(
    callbacks=ConfigurableField(
        id="callbacks",
        name="callbacks",
        description="A list of callbacks to use for streaming",
    )
)

We reinitialize our `agent`, nothing changes here:

In [50]:
# define the agent runnable
agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any")
)

Now, we will define our _custom_ callback handler. This will be a queue callback handler that will allow us to stream the output of the agent through an `asyncio.Queue` object and yield the tokens as they are generated elsewhere.

In [51]:
import asyncio
from langchain.callbacks.base import AsyncCallbackHandler


class QueueCallbackHandler(AsyncCallbackHandler):
    """Callback handler that puts tokens into a queue."""

    def __init__(self, queue: asyncio.Queue):
        self.queue = queue
        self.final_answer_seen = False

    async def __aiter__(self):
        while True:
            if self.queue.empty():
                await asyncio.sleep(0.1)
                continue
            token_or_done = await self.queue.get()

            if token_or_done == "<<DONE>>":
                # this means we're done
                return
            if token_or_done:
                yield token_or_done

    async def on_llm_new_token(self, *args, **kwargs) -> None:
        """Put new token in the queue."""
        #print(f"on_llm_new_token: {args}, {kwargs}")
        chunk = kwargs.get("chunk")
        if chunk:
            # check for final_answer tool call
            if tool_calls := chunk.message.additional_kwargs.get("tool_calls"):
                if tool_calls[0]["function"]["name"] == "final_answer":
                    # this will allow the stream to end on the next `on_llm_end` call
                    self.final_answer_seen = True
        self.queue.put_nowait(kwargs.get("chunk"))
        return

    async def on_llm_end(self, *args, **kwargs) -> None:
        """Put None in the queue to signal completion."""
        #print(f"on_llm_end: {args}, {kwargs}")
        # this should only be used at the end of our agent execution, however LangChain
        # will call this at the end of every tool call, not just the final tool call
        # so we must only send the "done" signal if we have already seen the final_answer
        # tool call
        if self.final_answer_seen:
            self.queue.put_nowait("<<DONE>>")
        else:
            self.queue.put_nowait("<<STEP_END>>")
        return

We can see how this works together in our `agent` invocation:

In [55]:
queue = asyncio.Queue()
streamer = QueueCallbackHandler(queue)

tokens = []

async def stream(query: str):
    response = agent.with_config(
        callbacks=[streamer]
    )
    async for token in response.astream({
        "input": query,
        "chat_history": [],
        "agent_scratchpad": []
    }):
        tokens.append(token)
        print(token, flush=True)

await stream("What is 57656756*6757655")

content='' additional_kwargs={} response_metadata={} id='run-35a522c7-7861-403c-93b6-5a916a18de00' tool_calls=[{'name': 'multiply', 'args': {'x': 57656756, 'y': 6757655}, 'id': '4d9ce070-013b-43be-82df-08234e26972b', 'type': 'tool_call'}] tool_call_chunks=[{'name': 'multiply', 'args': '{"x": 57656756, "y": 6757655}', 'id': '4d9ce070-013b-43be-82df-08234e26972b', 'index': None, 'type': 'tool_call_chunk'}]
content='' additional_kwargs={} response_metadata={} id='run-35a522c7-7861-403c-93b6-5a916a18de00' tool_calls=[{'name': 'final_answer', 'args': {'answer': 'The product of 57656756 and 6757655 is 3890214028816100.', 'tools_used': ['multiply']}, 'id': 'b3d860e2-855b-419c-827a-ac6e95d44c20', 'type': 'tool_call'}] tool_call_chunks=[{'name': 'final_answer', 'args': '{"answer": "The product of 57656756 and 6757655 is 3890214028816100.", "tools_used": ["multiply"]}', 'id': 'b3d860e2-855b-419c-827a-ac6e95d44c20', 'index': None, 'type': 'tool_call_chunk'}]
content='' additional_kwargs={} respon

In [57]:
tk = tokens[0]

for token in tokens[1:]:
    tk += token

tk

AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model': 'mistral', 'created_at': '2025-03-24T15:15:00.02675753Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6462483475, 'load_duration': 7733233, 'prompt_eval_count': 541, 'prompt_eval_duration': 8235911, 'eval_count': 265, 'eval_duration': 6443622717, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-35a522c7-7861-403c-93b6-5a916a18de00', tool_calls=[{'name': 'multiply', 'args': {'x': 57656756, 'y': 6757655}, 'id': '4d9ce070-013b-43be-82df-08234e26972b', 'type': 'tool_call'}, {'name': 'final_answer', 'args': {'answer': 'The product of 57656756 and 6757655 is 3890214028816100.', 'tools_used': ['multiply']}, 'id': 'b3d860e2-855b-419c-827a-ac6e95d44c20', 'type': 'tool_call'}], usage_metadata={'input_tokens': 541, 'output_tokens': 265, 'total_tokens': 806}, tool_call_chunks=[{'name': 'multiply', 'args': '{"x": 57656756, "y": 6757655}', 'id': '4d9ce070-013b-43be-82df-082

Now we're seeing that the output is being streamed token-by-token. Because we're being streamed a tool call the `content` field is empty. Instead, we can see the tokens being added inside the `tool_calls` fields, within `id`, `function.name`, and `function.arguments`.

In [95]:
from langchain_core.messages import ToolMessage

class CustomAgentExecutor:
    chat_history: list[BaseMessage]

    def __init__(self, max_iterations: int = 3):
        self.chat_history = []
        self.max_iterations = max_iterations
        self.agent: RunnableSerializable = (
            {
                "input": lambda x: x["input"],
                "chat_history": lambda x: x["chat_history"],
                "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
            }
            | prompt
            | llm.bind_tools(tools, tool_choice="any")  # we're forcing tool use again
        )

    async def invoke(self, input: str, streamer: QueueCallbackHandler, verbose: bool = False) -> dict:
        # invoke the agent but we do this iteratively in a loop until
        # reaching a final answer
        count = 0
        agent_scratchpad = []
        while count < self.max_iterations:
            # invoke a step for the agent to generate a tool call
            async def stream(query: str):
                response = self.agent.with_config(
                    callbacks=[streamer]
                )
                # we initialize the output dictionary that we will be populating with
                # our streamed output
                output = None
                # now we begin streaming
                async for token in response.astream({
                    "input": query,
                    "chat_history": self.chat_history,
                    "agent_scratchpad": agent_scratchpad
                }):
                    if output is None:
                        output = token
                    else:
                        # we can just add the tokens together as they are streamed and
                        # we'll have the full response object at the end
                        output += token
                    if token.content != "":
                        # we can capture various parts of the response object
                        if verbose: print(f"content: {token.content}", flush=True)
                    tool_calls = getattr(token, "tool_calls", None)
                    if tool_calls:
                        if verbose: print(f"tool_calls: {tool_calls}", flush=True)
                        tool_name = tool_calls[0]["name"]
                        if tool_name:
                            if verbose: print(f"tool_name: {tool_name}", flush=True)
                        arg = tool_calls[0]["args"]
                        if arg != "":
                            if verbose: print(f"arg: {arg}", flush=True)
                return AIMessage(
                    content=output.content,
                    tool_calls=output.tool_calls,
                    tool_call_id = output.tool_calls[0]["id"] if output.tool_calls else None

                )

            tool_call = await stream(query=input)
            if tool_call.tool_call_id==None:
                count += 1
                continue
            # add initial tool call to scratchpad
            agent_scratchpad.append(tool_call)
            # otherwise we execute the tool and add it's output to the agent scratchpad
            tool_name = tool_call.tool_calls[0]["name"]
            tool_args = tool_call.tool_calls[0]["args"]
            tool_call_id = tool_call.tool_call_id
            tool_out = name2tool[tool_name](**tool_args)
            # add the tool output to the agent scratchpad
            tool_exec = ToolMessage(
                content=f"{tool_out}",
                tool_call_id=tool_call_id
            )
            agent_scratchpad.append(tool_exec)
            count += 1
            # if the tool call is the final answer tool, we stop
            if tool_name == "final_answer":
                break
        # add the final output to the chat history, we only add the "answer" field
        if tool_out!=None:
          final_answer = str(tool_out)
        else:
          final_answer = tool_call.content
        self.chat_history.extend([
            HumanMessage(content=input),
            AIMessage(content=final_answer)
        ])
        # return the final answer in dict form
        return tool_args

agent_executor = CustomAgentExecutor()

We've added a few `print` statements to help us see what is being output, we activate those by setting `verbose=True`. Let's see what is returned:

In [96]:
queue = asyncio.Queue()
streamer = QueueCallbackHandler(queue)

out = await agent_executor.invoke("What is 67678*8687687687", streamer, verbose=True)

content:  To find the product of two large numbers, I will use the multiply tool. Here's how it works:

1. Use the multiply tool with 'x' as 67678 and 'y' as 8687687687.
2. The output from the multiply tool will be the product of the two numbers.
3. Finally, use the final_answer tool to provide a clear and concise answer to the user.

Here's the code:
```javascript
let result = add({x: multiply({x: 67678, y: 8687687687}, {x: 1, y: 1})[0], y: 0});
final_answer({answer: `The product of 67678 and 8687687687 is ${result}`, tools_used: ["multiply"]});
```
tool_calls: [{'name': 'multiply', 'args': {'x': 67678, 'y': 8687687687}, 'id': '66385917-86a2-44e8-9b98-bcd0c943e056', 'type': 'tool_call'}]
tool_name: multiply
arg: {'x': 67678, 'y': 8687687687}
tool_calls: [{'name': 'final_answer', 'args': {'answer': 'The product of the two numbers is [result from multiply tool]', 'tools_used': ['multiply']}, 'id': '51b99c0c-b530-42c4-bcbe-870a1a5d9183', 'type': 'tool_call'}]
tool_name: final_answer
arg:

We can see what is being output through the `verbose=True` flag. However, if we do _not_ `print` the output, we will see nothing:

In [97]:
queue = asyncio.Queue()
streamer = QueueCallbackHandler(queue)

out = await agent_executor.invoke("What is 67678*868767", streamer)

Although we see nothing, it does not mean that nothing is being returned to us - we're just not using our callback handler and `asyncio.Queue`. To use these we create an `asyncio` task, iterate over the `__aiter__` method of our `streamer` object, and await the task, like so:

In [ ]:
queue = asyncio.Queue()
streamer = QueueCallbackHandler(queue)

task = asyncio.create_task(agent_executor.invoke("What is 67678*868767", streamer))

async for token in streamer:
    print(token, flush=True)

await task

message=AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-4fa4fe1e-e512-4c3b-9082-637f0619c7aa', tool_calls=[{'name': 'multiply', 'args': {'x': 67678, 'y': 868767}, 'id': 'c95beb6f-e4ad-439f-bfbf-d23abb29dca9', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'multiply', 'args': '{"x": 67678, "y": 868767}', 'id': 'c95beb6f-e4ad-439f-bfbf-d23abb29dca9', 'index': None, 'type': 'tool_call_chunk'}])
message=AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-4fa4fe1e-e512-4c3b-9082-637f0619c7aa', tool_calls=[{'name': 'final_answer', 'args': {'answer': 'The product of 67678 and 868767 is 58796413026.', 'tools_used': ['multiply']}, 'id': 'dff11d58-d38e-4284-9b94-4f823d3b921f', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'final_answer', 'args': '{"answer": "The product of 67678 and 868767 is 58796413026.", "tools_used": ["multiply"]}', 'id': 'dff11d58-d38e-4284-9b94-4f823d3b921f', 'index': None, 'type': 'tool_call_chunk'}])
genera

Although this seems like a lot of work, we're now streaming tokens in a way that allows us to pass these tokens on to other parts of our code - such as through a websocket, streamed API response, or some downstream processing.

Let's try this out, we'll put together some simple post-processing to allow us to more nicely format the streamed output from out agent.

In [ ]:
queue = asyncio.Queue()
streamer = QueueCallbackHandler(queue)

task = asyncio.create_task(agent_executor.invoke("What is 10 + 10", streamer))

async for token in streamer:
    # first identify if we have a <<STEP_END>> token
    if token == "<<STEP_END>>":
        print("\n", flush=True)
    # we'll first identify if the token is a tool call
    elif tool_calls := token.message.additional_kwargs.get("tool_calls"):
        # if we have a tool call with a tool name, we'll print it
        if tool_name := tool_calls[0]["function"]["name"]:
            print(f"Calling {tool_name}...", flush=True)
        # if we have a tool call with arguments, we ad them to our args string
        if tool_args := tool_calls[0]["function"]["arguments"]:
            print(f"{tool_args}", end="", flush=True)

_ = await task

Calling add...
{"x":10,"y":10}

Calling final_answer...
{"answer":"10 + 10 equals 20.","tools_used":["functions.add"]}

With that we've produced a nice streaming output within our notebook - which ofcourse can be applied with very similar logic elsewhere, such as within a more polished web app.